In [57]:
# add TEI (and other) namespaces to tags and attributes
from lxml import etree

nsMap={
    None:'http://www.tei-c.org/ns/1.0',
    'tei':'http://www.tei-c.org/ns/1.0',
    'edx':'http://eudoxos.github.io/vism',
    'xinclude':'http://www.w3.org/2001/XInclude',
    'xml':'http://www.w3.org/XML/1998/namespace',
}

def tagNs(tag):
    if tag in ('TEI','teiHeader','text','front','main',
               'back','div','head','p','note','span',
               'table','cell','row','list','item',
               'lg','l','pb','ptr'): return None
    # if tag in ('id',): return 'xml'
    if tag in ('xinclude',): return 'xinclude'
    if tag in ('bibliography','bibentry','entry','index','glossary','em','IGNORE','introductory'): return 'edx'
    raise RuntimeError(f'Unable to find namespace for tag <{tag}>.')
def attrNs(att):
    if att in ('type','subtype','n','ed','rend','target'): return None
    if att in ('y','x','page_id','page_no','pdf_page','abbrev','loc','place','name','size','toc_name',
               'subtitle_pali','par_begin','par_end','anchor','reference_existing_footnote',
               'comment','start','title','subtitle'): return 'edx'
    if att in ('id',): return 'xml'
    raise RuntimeError(f'Unable to find namespace for attribute "{att}".')
def addNs(e):
    def _ns(s,t): return ('{'+s+'}' if s is not None else '{}')+t
    e.tag=_ns(nsMap[tagNs(e.tag)],e.tag)
    for a in list(e.attrib.keys()):
        pre=attrNs(a)
        if pre is None: continue
        e.attrib[_ns(nsMap[pre],a)]=e.attrib[a]
        del e.attrib[a]

        
    
#def attribNs(attr):
#    if 

def add_namespaces(fin,fout):
    doc0=etree.parse(fin,etree.XMLParser(remove_blank_text=True)).getroot() 
    # replace root element 
    doc=etree.Element('{}TEI',nsmap=nsMap)
    for e in doc0: doc.append(e)
    # add namespaces everywhere
    for e in doc.iter(): addNs(e)
    etree.cleanup_namespaces(doc,top_nsmap=nsMap)
    open(fout,'w').write(etree.tostring(doc,encoding='unicode',pretty_print=True))
    
add_namespaces('build/book.sectioned.tei','build/book.sectioned.namespaced.tei')
    
    
    

